In [ ]:
MOUNT_POINT = '/content/gdrive'
ROOT_PATH = MOUNT_POINT + "/My Drive/tfm"
DATA_PATH = ROOT_PATH + "/data"

import os
from google.colab import drive

drive.mount(MOUNT_POINT)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
samples_df = pd.read_parquet(f"{DATA_PATH}/samples.parquet")

In [ ]:
samples_df.head()

,GEMS Station Number,Sample Date,Sample Time,Depth,Parameter Code,Analysis Method Code,Value Flags,Value,Unit,Data Quality
0,ARG00010,1993-11-26,10:00,5.0,EC,EC-CND-Pt,None,30.00,µS/cm,Fair
1,ARG00010,1993-11-26,10:00,5.0,NOxN,COL-AZO-Cd,None,0.01,mg/l,Fair
2,ARG00010,1993-11-26,10:00,5.0,Mg-Dis,T-COL-EDTA-EBT,None,1.00,mg/l,Fair
3,ARG00010,1993-11-26,10:10,40.0,Ca-Dis,T-COL-EDTA-HNB,None,3.00,mg/l,Fair
4,ARG00010,1993-11-26,10:10,40.0,NOxN,COL-AZO-Cd,None,0.01,mg/l,Fair


In [ ]:
samples_tmp = samples_df.copy()
samples_tmp.head()

,GEMS Station Number,Sample Date,Sample Time,Depth,Parameter Code,Analysis Method Code,Value Flags,Value,Unit,Data Quality
0,ARG00010,1993-11-26,10:00,5.0,EC,EC-CND-Pt,None,30.00,µS/cm,Fair
1,ARG00010,1993-11-26,10:00,5.0,NOxN,COL-AZO-Cd,None,0.01,mg/l,Fair
2,ARG00010,1993-11-26,10:00,5.0,Mg-Dis,T-COL-EDTA-EBT,None,1.00,mg/l,Fair
3,ARG00010,1993-11-26,10:10,40.0,Ca-Dis,T-COL-EDTA-HNB,None,3.00,mg/l,Fair
4,ARG00010,1993-11-26,10:10,40.0,NOxN,COL-AZO-Cd,None,0.01,mg/l,Fair


In [ ]:
samples_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2498247 entries, 0 to 2498246
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   GEMS Station Number   object 
 1   Sample Date           object 
 2   Sample Time           object 
 3   Depth                 float64
 4   Parameter Code        object 
 5   Analysis Method Code  object 
 6   Value Flags           object 
 7   Value                 float64
 8   Unit                  object 
 9   Data Quality          object 
dtypes: float64(2), object(8)
memory usage: 190.6+ MB


In [ ]:
samples_tmp['Data Quality'].value_counts()

,count
Data Quality,
Fair,2498240
Good,7


In [ ]:
samples_tmp['Value Flags'].value_counts()

,count
Value Flags,
<,331699
>,409


In [ ]:
samples_tmp = samples_tmp[samples_tmp['Value Flags'].isna()]
samples_tmp.shape

(2166139, 10)

In [ ]:
samples_tmp.drop(
    columns=[
        'Sample Time',
        'Depth',
        'Analysis Method Code',
        'Value Flags',
        'Data Quality'
    ],
    inplace=True,
)

samples_tmp.head()

,GEMS Station Number,Sample Date,Parameter Code,Value,Unit
0,ARG00010,1993-11-26,EC,30.00,µS/cm
1,ARG00010,1993-11-26,NOxN,0.01,mg/l
2,ARG00010,1993-11-26,Mg-Dis,1.00,mg/l
3,ARG00010,1993-11-26,Ca-Dis,3.00,mg/l
4,ARG00010,1993-11-26,NOxN,0.01,mg/l


In [ ]:
TARGET_PARAMETERS = [
    "BOD",
    "Chl-a",
    "COD",
    "DIC",
    "DIN",
    "DIP",
    "DKN",
    "DOC",
    "DON",
    "DRP",
    "FDS",
    "FS",
    "NH3N",
    "NH4N",
    "NO2N",
    "NO3N",
    "NOxN",
    "O2-Dis",
    "O2-Dis-Sat",
    "pH",
    "PN",
    "POC",
    "PON",
    "PV",
    "SiO2-Dis",
    "SiO2-Rea",
    "SiO2-Tot",
    "TDN",
    "TDP",
    "TDS",
    "TEMP",
    "TEMP-Air",
    "TIC",
    "TIP",
    "TKN",
    "TN",
    "TOC",
    "TON",
    "TP",
    "TPP",
    "TRANS",
    "TRP",
    "TS",
    "TSS",
    "TURB",
    "VDS",
    "VS",
]

In [ ]:
samples_target = samples_tmp[samples_tmp['Parameter Code'].isin(TARGET_PARAMETERS)]
samples_target = samples_target.dropna(subset=['Value'])
samples_target.reset_index(drop=True, inplace=True)
samples_target.head()

,GEMS Station Number,Sample Date,Parameter Code,Value,Unit
0,ARG00010,1993-11-26,NOxN,0.010,mg/l
1,ARG00010,1993-11-26,NOxN,0.010,mg/l
2,ARG00010,1994-03-10,O2-Dis,9.800,mg/l
3,ARG00010,1994-03-10,NOxN,0.020,mg/l
4,ARG00010,1994-03-10,TP,0.001,mg/l


In [ ]:
samples_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803779 entries, 0 to 803778
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   GEMS Station Number  803779 non-null  object 
 1   Sample Date          803779 non-null  object 
 2   Parameter Code       803779 non-null  object 
 3   Value                803779 non-null  float64
 4   Unit                 803779 non-null  object 
dtypes: float64(1), object(4)
memory usage: 30.7+ MB


In [ ]:
counter = samples_target['Parameter Code'].value_counts()
counter_df = counter.reset_index()
counter_df.columns = ['Parameter Code', 'Count']
counter_df

,Parameter Code,Count
0,TEMP,76351
1,O2-Dis,66476
2,TP,65302
3,pH,58505
4,NO3N,53167
5,O2-Dis-Sat,52602
6,NH4N,51963
7,NO2N,44002
8,TN,34062
9,Chl-a,33558


In [ ]:
sub_n = counter_df[counter_df['Parameter Code'].isin(["DIN", "DKN", "DON", "NH3N", "NH4N", "NO2N", "NO3N", "NOxN", "PN", "PON", "TDN", "TKN", "TN", "TON"])]
sub_n

,Parameter Code,Count
4,NO3N,53167
6,NH4N,51963
7,NO2N,44002
8,TN,34062
13,TKN,19874
17,NH3N,16231
19,NOxN,13400
29,TON,4157
33,DON,1314
34,TDN,728


In [ ]:
sub_p = counter_df[counter_df['Parameter Code'].isin(["DIP", "DRP", "TDP", "TIP", "TP", "TPP", "TRP"])]
sub_p

,Parameter Code,Count
2,TP,65302
10,TIP,28630
20,DIP,9771
21,DRP,9504
22,TDP,9422
30,TRP,3873
32,TPP,1536


In [ ]:
sub_o = counter_df[counter_df['Parameter Code'].isin(["O2-Dis", "BOD", "COD", "PV"])]
sub_o

,Parameter Code,Count
1,O2-Dis,66476
25,COD,6968
26,BOD,6426
31,PV,1589


In [ ]:
sub_f = counter_df[counter_df['Parameter Code'].isin(["TRANS", "TURB", "TEMP", "FDS", "FS", "TDS", "TS", "TSS", "VDS", "VS"])]
sub_f

,Parameter Code,Count
0,TEMP,76351
11,TRANS,22246
12,TSS,20421
15,TURB,16700
16,TDS,16346
42,FDS,21
43,FS,21
44,TS,21
45,VS,15
46,VDS,4


In [ ]:
FINAL_PARAMETERS = [
    "Chl-a",
    "DIN",
    "DKN",
    "NH3N",
    "NH4N",
    "NOxN",
    "NO2N",
    "NO3N",
    "PN",
    "PON",
    "TDN",
    "TKN",
    "TN",
    "TON",
    "DON",
    "DIP",
    "DRP",
    "TDP",
    "TIP",
    "TP",
    "TRP",
    "TPP",
    "BOD",
    "COD",
    "O2-Dis",
    "PV",
    "TDS",
    "TS",
    "TSS",
    "FDS",
    "FS",
    "VDS",
    "VS",
    "TRANS",
    "TURB",
    "TEMP",
    "pH",
]

In [ ]:
samples_final = samples_tmp[samples_tmp['Parameter Code'].isin(FINAL_PARAMETERS)]
samples_final = samples_final.dropna(subset=['Value'])
samples_final.reset_index(drop=True, inplace=True)
samples_final.head()

,GEMS Station Number,Sample Date,Parameter Code,Value,Unit
0,ARG00010,1993-11-26,NOxN,0.010,mg/l
1,ARG00010,1993-11-26,NOxN,0.010,mg/l
2,ARG00010,1994-03-10,O2-Dis,9.800,mg/l
3,ARG00010,1994-03-10,NOxN,0.020,mg/l
4,ARG00010,1994-03-10,TP,0.001,mg/l


In [ ]:
samples_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693514 entries, 0 to 693513
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   GEMS Station Number  693514 non-null  object 
 1   Sample Date          693514 non-null  object 
 2   Parameter Code       693514 non-null  object 
 3   Value                693514 non-null  float64
 4   Unit                 693514 non-null  object 
dtypes: float64(1), object(4)
memory usage: 26.5+ MB


In [ ]:
units = samples_final.groupby('Parameter Code')['Unit'].value_counts()
units_df = units.reset_index(name='Count')
units_df

,Parameter Code,Unit,Count
0,BOD,mg/l,6426
1,COD,mg/l,6968
2,Chl-a,mg/l,31854
3,Chl-a,µg/l,1704
4,DIN,mg/l,57
5,DIP,mg/l,9771
6,DKN,mg/l,423
7,DON,mg/l,1314
8,DRP,mg/l,7678
9,DRP,µg/l,1826


In [ ]:
# Identificar las filas donde la unidad es 'µg/g'
filtro_conversion = samples_final['Unit'].isin(['µg/g'])

# Corregir la unidad
samples_final.loc[filtro_conversion, 'Unit'] = 'µg/l'

# Identificar las filas donde la unidad es 'µg/l'
filtro_conversion = samples_final['Unit'].isin(['µg/l'])

# Realizar la conversión de 'Value' de ug/l a mg/l
samples_final.loc[filtro_conversion, 'Value'] = samples_final.loc[filtro_conversion, 'Value'] / 1000

# Identificar las columnas con nombre difernete
filtro_conversion = samples_final['Unit'].isin(['µg/l', 'mg/l NO3', 'mg/l NH4', 'mg/l NO2'])

# Cambiar la unidad a 'mg/l'
samples_final.loc[filtro_conversion, 'Unit'] = 'mg/l'

# Identificar files con unidades incompatibles
filtro_conversion = samples_final['Unit'].isin(['%', 'JCU'])

# Eliminar filas incompatibles
samples_final = samples_final[~filtro_conversion]

# Resetear el índice después de eliminar las filas (opcional)
samples_final.reset_index(drop=True, inplace=True)

In [ ]:
units = samples_final.groupby('Parameter Code')['Unit'].value_counts()
display(units)

,,count
Parameter Code,Unit,
BOD,mg/l,6426
COD,mg/l,6968
Chl-a,mg/l,33558
DIN,mg/l,57
DIP,mg/l,9771
DKN,mg/l,423
DON,mg/l,1314
DRP,mg/l,9504
FDS,mg/l,21


In [ ]:
samples_final.drop(
    columns=[
        'Unit'
    ],
    inplace=True,
)
samples_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692587 entries, 0 to 692586
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   GEMS Station Number  692587 non-null  object 
 1   Sample Date          692587 non-null  object 
 2   Parameter Code       692587 non-null  object 
 3   Value                692587 non-null  float64
dtypes: float64(1), object(3)
memory usage: 21.1+ MB


In [ ]:
samples_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692587 entries, 0 to 692586
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   GEMS Station Number  692587 non-null  object 
 1   Sample Date          692587 non-null  object 
 2   Parameter Code       692587 non-null  object 
 3   Value                692587 non-null  float64
dtypes: float64(1), object(3)
memory usage: 21.1+ MB


In [ ]:
# Pivotear el DataFrame para que los parámetros sean columnas
df_pivot = samples_final.pivot_table(
    index=['GEMS Station Number', 'Sample Date'],
    columns='Parameter Code',
    values='Value',
    aggfunc='first'
)

# Resetear el índice para convertirlo en columnas normales
df_pivot.columns.name = None
df_pivot = df_pivot.reset_index()

# Opcional: Reorganizar las columnas para que GEMS Station Number y Sample Date estén al inicio
samples_table = df_pivot[['GEMS Station Number', 'Sample Date'] + FINAL_PARAMETERS]

# Renombrar columnas Chl-a y O2-Dis
samples_table.rename(columns={'Chl-a': 'Chl_a', 'O2-Dis': 'O2_Dis'}, inplace=True)

samples_table.head()

,GEMS Station Number,Sample Date,Chl_a,DIN,DKN,NH3N,NH4N,NOxN,NO2N,NO3N,...,TS,TSS,FDS,FS,VDS,VS,TRANS,TURB,TEMP,pH
0,ARG00010,1993-11-26,0.0001,NaN,NaN,0.01,NaN,0.01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,9.0,7.3
1,ARG00010,1994-03-10,0.3930,NaN,NaN,NaN,NaN,0.02,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,15.2,6.2
2,ARG00010,1994-11-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19.5,NaN,9.5,7.2
3,ARG00010,1995-03-30,0.0058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,9.2,6.6
4,ARG00010,1995-09-26,0.0080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,8.3,6.8


In [ ]:
samples_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70508 entries, 0 to 70507
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   GEMS Station Number  70508 non-null  object 
 1   Sample Date          70508 non-null  object 
 2   Chl_a                30295 non-null  float64
 3   DIN                  57 non-null     float64
 4   DKN                  165 non-null    float64
 5   NH3N                 14698 non-null  float64
 6   NH4N                 38901 non-null  float64
 7   NOxN                 11951 non-null  float64
 8   NO2N                 31987 non-null  float64
 9   NO3N                 39274 non-null  float64
 10  PN                   311 non-null    float64
 11  PON                  106 non-null    float64
 12  TDN                  303 non-null    float64
 13  TKN                  17367 non-null  float64
 14  TN                   29053 non-null  float64
 15  TON                  4157 non-null  

In [ ]:
samples_table.to_parquet(f'{DATA_PATH}/samples_final.parquet')
samples_table.to_csv(f'{DATA_PATH}/samples_final.csv', index=False)

In [ ]:
station_chl_a_stats = samples_table.groupby('GEMS Station Number')['Chl_a'].agg(['count', 'size', lambda x: (x.notna().sum() / x.size) * 100])
station_chl_a_stats = station_chl_a_stats.rename(columns={'<lambda_0>': '% Not Null', 'count': 'Not Null Count', 'size': 'Total Count'})
station_chl_a_stats = station_chl_a_stats.sort_values(by=['Not Null Count'], ascending=False)
station_chl_a_stats

,Not Null Count,Total Count,% Not Null
GEMS Station Number,,,
URY00076,3488,5593,62.363669
JPN00025,470,561,83.778966
JPN00026,458,548,83.576642
JPN00027,424,512,82.812500
NOR00019,384,529,72.589792
...,...,...,...
CAN00268,0,59,0.000000
CAN00269,0,54,0.000000
CAN00270,0,60,0.000000


In [ ]:
TARGET_NITROGEN_VAR = 'TRANS'                                                    # NO3N, NH4N, NO2N, TN
station_no3n_stats = samples_table.groupby('GEMS Station Number')[TARGET_NITROGEN_VAR].agg(['count', 'size', lambda x: (x.notna().sum() / x.size) * 100])
station_no3n_stats = station_no3n_stats.rename(columns={'<lambda_0>': '% Not Null', 'count': 'Not Null Count', 'size': 'Total Count'})
station_no3n_stats = station_no3n_stats.sort_values(by=['Not Null Count'], ascending=False)
station_no3n_stats

,Not Null Count,Total Count,% Not Null
GEMS Station Number,,,
JPN00025,479,561,85.383244
JPN00026,461,548,84.124088
JPN00027,425,512,83.007812
CAN00010,356,565,63.008850
NOR00019,332,529,62.759924
...,...,...,...
DEU01632,0,81,0.000000
DEU01633,0,18,0.000000
DEU01636,0,241,0.000000


In [ ]:
TARGET_PHOSPHATE_VAR = 'TP'                                                     # TP, TIP, DIP, DRP
station_tp_stats = samples_table.groupby('GEMS Station Number')[TARGET_PHOSPHATE_VAR].agg(['count', 'size', lambda x: (x.notna().sum() / x.size) * 100])
station_tp_stats = station_tp_stats.rename(columns={'<lambda_0>': '% Not Null', 'count': 'Not Null Count', 'size': 'Total Count'})
station_tp_stats = station_tp_stats.sort_values(by=['Not Null Count'], ascending=False)
station_tp_stats

,Not Null Count,Total Count,% Not Null
GEMS Station Number,,,
NLD00347,518,529,97.920605
NOR00019,513,529,96.975425
JPN00025,487,561,86.809269
JPN00026,475,548,86.678832
NLD00323,444,444,100.000000
...,...,...,...
RUS00025,0,115,0.000000
RUS00026,0,149,0.000000
RUS00044,0,1,0.000000


In [ ]:
water_body_ids = {
    'CAN00010': 'LAGO ONTARIO',
    'CAN00044': 'LAGO ONTARIO',
    'CAN00045': 'LAGO ONTARIO',
    'CAN00046': 'LAGO ONTARIO',
    'CAN00047': 'LAGO ONTARIO',
    'CAN00048': 'LAGO ONTARIO',
    'JPN00025': 'LAGO KASUMIGAURA',
    'JPN00026': 'LAGO KASUMIGAURA',
    'JPN00027': 'LAGO KASUMIGAURA',
    'NOR00004': 'LAGO MJOSA',
    'NOR00019': 'LAGO MJOSA',
    'NOR00037': 'LAGO MJOSA',
    'NOR00050': 'LAGO MJOSA',
    'NOR00051': 'LAGO MJOSA',
    'LVA00003': 'LAGO MAZAIS BALTEZERS',
    'MEX01303': 'LAGO ESTERO DE MEZCALA',
    'MEX01304': 'LAGO ESTERO DE MEZCALA',
    'MEX01305': 'LAGO ESTERO DE MEZCALA',
    'MEX01306': 'LAGO ESTERO DE MEZCALA',
}


In [ ]:
samples_table_filtered = samples_table[samples_table['GEMS Station Number'].isin(water_body_ids.keys())].copy()
samples_table_filtered['Water Body'] = samples_table_filtered['GEMS Station Number'].map(water_body_ids)
cols = ['Water Body'] + [col for col in samples_table_filtered if col != 'Water Body']
samples_table_filtered = samples_table_filtered[cols]
samples_table_filtered.drop(columns=['GEMS Station Number'], inplace=True)
samples_table_filtered.reset_index(drop=True, inplace=True)
samples_table_filtered.head()

,Water Body,Sample Date,Chl_a,DIN,DKN,NH3N,NH4N,NOxN,NO2N,NO3N,...,TS,TSS,FDS,FS,VDS,VS,TRANS,TURB,TEMP,pH
0,LAGO ONTARIO,1979-03-22,0.0015,NaN,NaN,NaN,NaN,0.34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,0.8,8.0
1,LAGO ONTARIO,1979-04-12,0.0020,NaN,NaN,NaN,NaN,0.34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,1.5,8.3
2,LAGO ONTARIO,1979-05-03,0.0027,NaN,NaN,NaN,NaN,0.34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1,8.0
3,LAGO ONTARIO,1979-05-29,0.0016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,3.1,8.2
4,LAGO ONTARIO,1979-06-26,0.0066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,10.1,8.2


In [ ]:
samples_table_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3472 entries, 0 to 3471
Data columns (total 39 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Water Body   3472 non-null   object 
 1   Sample Date  3472 non-null   object 
 2   Chl_a        2401 non-null   float64
 3   DIN          0 non-null      float64
 4   DKN          60 non-null     float64
 5   NH3N         1554 non-null   float64
 6   NH4N         136 non-null    float64
 7   NOxN         1666 non-null   float64
 8   NO2N         195 non-null    float64
 9   NO3N         1061 non-null   float64
 10  PN           61 non-null     float64
 11  PON          83 non-null     float64
 12  TDN          0 non-null      float64
 13  TKN          97 non-null     float64
 14  TN           2215 non-null   float64
 15  TON          105 non-null    float64
 16  DON          959 non-null    float64
 17  DIP          75 non-null     float64
 18  DRP          1895 non-null   float64
 19  TDP   

In [ ]:
samples_table_filtered.to_parquet(f'{DATA_PATH}/samples_refined_water_bodies.parquet')
samples_table_filtered.to_csv(f'{DATA_PATH}/samples_refined_water_bodies.csv', index=False)